In [1]:
import numpy as np
import copy

In [2]:
def bin_to_pattern(x):
    return [ [1-int(i), int(i)] for i in x ]

In [3]:
def create_specific_patterns(mode1,mode2,size):
    pats =[]
    for i in range(2):
        for j in range(2):
            pats.append( [[1,1]]*mode1 + bin_to_pattern(str(i)) + [[1,1]]*(mode2-mode1-1) + bin_to_pattern(str(j)) + [[1,1]]*(size-mode1-(mode2-mode1-1)-2) )
    return pats

In [4]:
def create_specific_patterns3(mode1,mode2,mode3,size):
    pats =[]
    for i in range(2):
        for j in range(2):
            for k in range(2):
                pats.append( [[1,1]]*mode1 + bin_to_pattern(str(i)) + [[1,1]]*(mode2-mode1-1) + bin_to_pattern(str(j)) + [[1,1]]*(mode3-(mode2-mode1-1)-mode1-2) + bin_to_pattern(str(k)) + [[1,1]]*(size -mode1-(mode2-mode1-1)-(mode3-(mode2-mode1-1)-mode1-2)-3) )
    return pats

In [5]:
def create_specific_patterns4(mode1,mode2,mode3,mode4,size):
    pats = []
    temp_pats = create_specific_patterns3(mode1,mode2,mode3,size)
    for p in temp_pats:
        for k in range(2):
            p_prime = copy.deepcopy(p)
            p_prime[mode4] = bin_to_pattern(str(k))[0]
            pats.append(p_prime)
    return pats
# create_specific_patterns4(0,1,2,4,N)

In [6]:
def dev_pattern(pat):
    f = np.array([1])
    for i in pat:
        f = np.kron(f,i)
    return f

In [7]:
N = 2
s = 2                       #Sparsity

all_pat = create_specific_patterns(0,1,N)
print((all_pat))

A = np.matrix([ dev_pattern(p) for p in all_pat])
print(A)
np.linalg.matrix_rank(A)

[[[1, 0], [1, 0]], [[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, 1], [0, 1]]]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


4

In [10]:
N = 6
all_pat = []
# all_pat = create_specific_patterns(0,1,print(result)N) + create_specific_patterns(2, 3,N) + create_specific_patterns(1, 3,N)
for i in range(N-1):
    all_pat += create_specific_patterns(i,i+1,N)

A = np.matrix([ dev_pattern(p) for p in all_pat])
A.shape

(20, 64)

In [30]:
N = 8
all_pat = []

l = np.arange(0,N)
all_pat = []
for i in range(8):
    np.random.shuffle(l)
    r = l[0:4]
    r.sort()
    all_pat += create_specific_patterns4(r[0],r[1],r[2],r[3],N)

# all_pat = create_specific_patterns4(0,1,3,5,N) + create_specific_patterns4(1,2,4,7,N)

A = np.matrix([ dev_pattern(p) for p in all_pat])
A.shape

(128, 256)

<h2>Coherence</h2>

In [14]:
def compute_coherence(A):
    M,N = A.shape
    coherence = 0
    mem = (0,0)
    for i in range(N):
        for j in range(i+1,N):
            temp = (A[:,i].T*A[:,j])[0,0]
            if temp>coherence:
                coherence = temp
                mem = (i,j)
    print("Bound {}".format(np.sqrt((N-M)/(M*(N-1)))))
    # print(mem)
    return coherence

print(compute_coherence(A))

Bound 0.1868706368604627
4


In [25]:
A = np.matrix(np.random.rand(32, 256))
compute_coherence(A)

Bound 0.1656833739159028


13.045676786504469

<h2>NSP</h2>

In [11]:
def combinations(arr, k):
    # Base case: if k is 0, return an empty list (one valid combination)
    if k == 0:
        return [[]]
    
    # If no more elements or not enough elements to form a combination, return empty list
    if len(arr) < k:
        return []
    
    # Take the first element
    first = arr[0]
    
    # Combinations that include the first element
    with_first = combinations(arr[1:], k - 1)
    
    # Add the first element to each of the combinations
    with_first = [[first] + comb for comb in with_first]
    
    # Combinations that exclude the first element
    without_first = combinations(arr[1:], k)
    
    # Return both with and without the first element
    return with_first + without_first

In [31]:
def spark(A):
    M,N = A.shape
    rk = np.linalg.matrix_rank(A)
    arr = np.arange(N)
    print("Bound {}".format(rk+1))
    for k in range(2,rk):
        print(k)
        all_combi = combinations(arr, k)
        for combi in all_combi:
            sub_A = A[:,combi]
            QA_sub_A = np.linalg.qr(sub_A)[1].round()
            if 0 in np.diag(QA_sub_A):
                # print(sub_A)
                print(combi)
                return k
    return rk+1

spark(A)

Bound 69
2
3
4
